In [1]:
import matplotlib.pyplot as plt 
import matplotlib.image as mtimage 
import numpy as np 
%matplotlib inline 
print(np.__version__)

1.22.4


In [107]:
# Imports
import cv2

# Canny Edge Detection : Convert to Gradient Image 
def canny(image):
    # Convert to Gray Image 
    gray_image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    # Blur or Smoothen the Image to reduce noise 
    kernel_size = 5
    blur_image = cv2.GaussianBlur(gray_image,(kernel_size,kernel_size),0)
    # Apply Canny Edge Detection Algorithm :
    # Reject values below low_threshold and accept above the high_threshold 
    # value in between low and high are only accepted if the there neighbouring 
    # pixel is above high_threshold
    low_threshold = 50
    high_threshold = 150
    canny_image = cv2.Canny(blur_image,low_threshold,high_threshold)
    
    return canny_image

# Region Selection : Select the region of interest 
def region_selection(image):
    # Create a Black Image with same shape 
    mask = np.zeros_like(image)
    # Color of the region to be selected 
    shape_color = 255
    # Create a Shape around the lane lines 
    vertices = np.array([[(160,720),(680,380),(1200,720)]])
    # Draw shape on the mask image 
    cv2.fillPoly(mask,vertices,shape_color)
    # Crop the image to only the region of interest 
    masked_image = cv2.bitwise_and(image,mask)
    return masked_image
# Hough Transformation : Detect Edges or Lines  
def hough_transformation(image):
    # Define Parameter 
    rho = 2 
    theta = np.pi / 180
    threshold = 100
    min_line_len = 100
    max_line_gap = 5
    # Return list of lines detected 
    return cv2.HoughLinesP(image,rho,theta,threshold,np.array([]),min_line_len,max_line_gap)

def draw_lane_lines(image,lines):
    # Create a black image with same shape 
    lines_image = np.zeros_like(image)
    # Draw lines on the black image 
    if lines is not None :
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
            cv2.line(lines_image,(x1,y1),(x2,y2),(0,0,255),15)
    return lines_image

# Make the coordinates for slope and intercept 
def get_coordinates(image,line_parameters):
    
    slope , intercept = line_parameters
    # Get the shape of the image 
    shape = image.shape
    # Set the y coord 
    # Set the start point to the bottom of the image 
    # the end point to the 3/5 th of the image from the bottom 
    y1 = shape[0]
    y2 = int(y1 * (3/5))
    # y = mx + c => x = (y-c)/m 
    # Calculate the x coordinates 
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])
    

def average_slopes(image,lines):
    left_fit = []
    right_fit = [] 
    # If some lines found
    if lines is  not None :
        for line in lines :
            x1,y1,x2,y2 = line.reshape(4)
            # Calculate the slope and intercept of the coord 
            slope, intercept = np.polyfit((x1,x2),(y1,y2),1)
            # if negative slope the line is on left lane 
            if slope < 0:
                left_fit.append((slope,intercept))
            # otherise line is on right lane
            else:
                right_fit.append((slope,intercept))
    # Calculate the average values for left and right lines
    if len(left_fit) == 0 :
        left_line = np.array([0,0,0,0])
    else:
        left_fit_average = np.average(left_fit,axis=0)
        left_line = get_coordinates(image,left_fit_average)
    if len(right_fit) == 0 :
        right_line = np.array([0,0,0,0])
    else:
        right_fit_average = np.average(right_fit,axis=0)
        right_line = get_coordinates(image,right_fit_average)
    return [left_line,right_line]
    
    
# Algorithm for Lane Detection in an image 
def process_image(image):
    laneline_image = np.copy(image)
    canny_image = canny(laneline_image)
    cropped_image = region_selection(canny_image)
    lines = hough_transformation(cropped_image)
    average_lines = average_slopes(image,lines)
    lines_image = draw_lane_lines(laneline_image,average_lines)
    combo_image = cv2.addWeighted(laneline_image,0.8,lines_image,1,0)
    return combo_image


capture = cv2.VideoCapture("test.mp4")
output_filename = "output.avi"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
output_video = cv2.VideoWriter(output_filename,fourcc,20.0,(shape[1],shape[0]))

# success,frame = capture.read()
# frame = process_image(frame)
# plt.imshow(frame)
# plt.show()


while capture.isOpened():
    success,frame = capture.read()
    if not success:
        break
    frame = process_image(frame)
    output_video.write(frame)
    cv2.imshow("testvideo",frame)
    key = cv2.waitKey(1)
    if key == ord("s"):
        break
    
capture.release()
output_video.release()
cv2.destroyAllWindows()


ValueError: not enough values to unpack (expected 4, got 1)